# Library import

### import librerie

In [1]:
import os
import numpy as np

import json
from PIL import Image
from torch.utils.data import Dataset

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm


In [2]:
pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


## Path

In [3]:
# file contenente i path delle immagini del dataset
txt_file = "/kaggle/input/our-xview-dataset/xView_class_map.json"
img_dir = "/kaggle/input/our-xview-dataset/images"

annotation_file = "/kaggle/input/our-xview-dataset/COCO_annotations_new.json"

In [4]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils', trust_repo=True)

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


# Dataloader

In [5]:
class CustomDataset(Dataset):
    def __init__(self, annotations_file, img_dir, utils, aug=False):
        """
        Args:
            annotations_file (str): Path al file JSON delle annotazioni (es. formato COCO).
            img_dir (str): Path alla directory delle immagini.
            transform (callable, optional): Trasformazioni da applicare alle immagini.
        """

        with open(annotations_file, 'r') as f:
            self.annotations = json.load(f)
        self.img_dir = img_dir
        self.utils = utils
        self.aug = aug

    def __len__(self):
        return len(self.annotations['images'])

    def __getitem__(self, idx):
        # Leggi i dettagli dell'immagine
        img_info = self.annotations['images'][idx]
        img_path = f"{self.img_dir}/{img_info['file_name']}"
        image = Image.open(img_path).convert("RGB")
        
        # Leggi le annotazioni
        img_id = img_info['id']
        annotations = [ann for ann in self.annotations['annotations'] if ann['image_id'] == img_id]
        bboxes = np.array([ann['bbox'] for ann in annotations], dtype=np.float32)
        labels = np.array([ann['category_id'] for ann in annotations], dtype=np.int64)
        
        # Converti bboxes nel formato richiesto (x_min, y_min, x_max, y_max)
        bboxes[:, 2:] += bboxes[:, :2]  # width, height -> x_max, y_max
        
        target = {
            'boxes': torch.tensor(bboxes, dtype=torch.float32),
            'labels': torch.tensor(labels, dtype=torch.int64),
        }
        
        #if self.aug:
            #image = self.transform(image)
        
        return image, target


In [6]:
def collate_fn(batch):

    # Estrai le immagini e i percorsi dal batch
    images = [item["image"] for item in batch]
    paths = [item["path"] for item in batch]
    
    # Combina i tensori delle immagini in un unico tensore
    # (assumendo che tutte le immagini abbiano la stessa dimensione dopo il preprocessing)
    images_tensor = torch.cat(images, dim=0)
    
    return {
        "images": images_tensor,  # Batch di immagini come tensore
        "paths": paths            # Percorsi originali delle immagini
    }


In [7]:
# Creazione dei dataset
train_dataset = CustomDataset(annotation_file, img_dir, utils, aug=True) 
valid_dataset = CustomDataset(annotation_file, img_dir, utils, aug=False)  
test_dataset = CustomDataset(annotation_file, img_dir, utils, aug=False)  

# Creazione dei DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Network

In [8]:
class SSDModel(nn.Module):
    def __init__(self, num_classes):
        super(SSDModel, self).__init__()

        ## Model -> per info https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Detection/SSD
        self.ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd') # modello pre-addestrato su dataset COCO

    def forward(self, images):

        # Calcola le previsioni con il modello SSD
        predictions = self.ssd_model(images)  # Output grezzo del modello SSD
        
        return predictions

In [9]:
class Trainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimizer=None, device='cuda'):
        """
        Inizializza la classe Trainer.
        
        Args:
            model: Il modello SSD.
            train_loader: DataLoader per il training set.
            val_loader: DataLoader per il validation set.
            criterion: Funzione di perdita.
            optimizer: Ottimizzatore (opzionale, di default Adam).
            device: Dispositivo per il calcolo ('cuda' o 'cpu').
        """
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer if optimizer else Adam(model.parameters(), lr=1e-4)
        self.device = device

    def train_one_epoch(self, epoch):
        """
        Esegue un'epoca di addestramento.
        """
        self.model.train()
        running_loss = 0.0
        pbar = tqdm(self.train_loader, desc=f"Training Epoch {epoch}")
        
        for images, targets in pbar:
            images, targets = images.to(self.device), targets.to(self.device)
            
            # Forward pass
            predictions = self.model(images)
            
            # Calcolo della perdita
            loss = self.criterion(predictions, targets)
            
            # Backward pass e ottimizzazione
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            running_loss += loss.item()
            pbar.set_postfix({"loss": running_loss / len(self.train_loader)})
        
        return running_loss / len(self.train_loader)
    
    def validate_one_epoch(self, epoch):
        """
        Esegue un'epoca di validazione.
        """
        self.model.eval()
        running_loss = 0.0
        pbar = tqdm(self.val_loader, desc=f"Validation Epoch {epoch}")
        
        with torch.no_grad():
            for images, targets in pbar:
                images, targets = images.to(self.device), targets.to(self.device)
                
                # Forward pass
                predictions = self.model(images)
                
                # Calcolo della perdita
                loss = self.criterion(predictions, targets)
                running_loss += loss.item()
                
                pbar.set_postfix({"val_loss": running_loss / len(self.val_loader)})
        
        return running_loss / len(self.val_loader)
    
    def fit(self, epochs):
        """
        Esegue l'addestramento e la validazione per un dato numero di epoche.
        """
        train_losses = []
        val_losses = []
        
        for epoch in range(1, epochs + 1):
            train_loss = self.train_one_epoch(epoch)
            val_loss = self.validate_one_epoch(epoch)
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            
            print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        return train_losses, val_losses

# vedere se è necessario inserire dei checkpoint durante il train

# Training

In [10]:
num_classes = 12
ssd_model = SSDModel(num_classes)

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 172MB/s] 
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Detection/SSD/ssd/entrypoints.py:201: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `t

In [14]:
# Addestra il modello, con validazione ad ogni epoca
loss = 
ssd_model.train(train_loader, val_loader, loss, optimizer= "Adam")

NameError: name 'criterion' is not defined

# Testing